In [3]:
import os, sys
import json
import numpy as np
import torch
from torch import nn, optim
from torch.optim import lr_scheduler

from opts import parse_opts
from model import generate_model
import transforms 
from dataset import get_training_set, get_validation_set, get_test_set
from utils import Logger, adjust_learning_rate, save_checkpoint
from train import train_epoch
from validation import val_epoch
import time

In [4]:
sys.argv

['/home/majidh/anaconda3/envs/mml/lib/python3.8/site-packages/ipykernel_launcher.py',
 '-f',
 '/home/majidh/.local/share/jupyter/runtime/kernel-ce4a7848-ee56-48b1-8d2b-2d73af163e73.json']

In [9]:
model = ''
dataset = ''
sample_duration = ''

sys.argv = ['main.py']

# Parsing command line option
opt = parse_opts()
print(opt)

n_folds = 1
test_accuracies = []

#Opt between CPU or GPU 
if opt.device != 'cpu':
    opt.device = 'cuda' if torch.cuda.is_available() else 'cpu'  

# specifies that the model is pretrained or not
pretrained = opt.pretrain_path != 'None'    

#make results folder 
if not os.path.exists(opt.result_path):
    os.makedirs(opt.result_path)

opt.arch = '{}'.format(opt.model)  
opt.store_name = '_'.join([opt.dataset, opt.model, str(opt.sample_duration)])

Namespace(annotation_path='ravdess_preprocessing/annotations.csv', batch_size=8, begin_epoch=1, dampening=0.9, dataset='RAVDESS', device='cuda', fusion='ia', learning_rate=0.04, lr_patience=10, lr_steps=[40, 55, 65, 70, 200, 250], manual_seed=1, mask='softhard', model='multimodalcnn', momentum=0.9, n_classes=8, n_epochs=100, n_threads=16, no_train=False, no_val=False, num_heads=1, pretrain_path='EfficientFace_Trained_on_AffectNet7.pth.tar', result_path='results', resume_path='', sample_duration=15, sample_size=224, store_name='model', test=True, test_subset='test', video_norm_value=255, weight_decay=0.001)


In [8]:
for fold in range(n_folds):

    print(opt)
    with open(os.path.join(opt.result_path, 'opts'+str(time.time())+str(fold)+'.json'), 'w') as opt_file:
        json.dump(vars(opt), opt_file)

    torch.manual_seed(opt.manual_seed)

    ################IMPORTANT THE MODEL AND PARAMETERS are put here IMPORTANT#################
    model, parameters = generate_model(opt)

    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(opt.device)

    if not opt.no_train:

        #define training data
        training_data = get_training_set(opt, spatial_transform=None) # we change the spatial transform to none to prevent generating random noise  

        train_loader = torch.utils.data.DataLoader(
            training_data,
            batch_size=opt.batch_size,
            shuffle=True,
            num_workers=opt.n_threads,
            pin_memory=True)

        train_logger = Logger(
            os.path.join(opt.result_path, 'train'+str(fold)+'.log'),
            ['epoch', 'loss', 'prec1', 'prec5', 'lr'])
        train_batch_logger = Logger(
            os.path.join(opt.result_path, 'train_batch'+str(fold)+'.log'),
            ['epoch', 'batch', 'iter', 'loss', 'prec1', 'prec5', 'lr'])


        optimizer = optim.SGD(
            parameters,
            lr=opt.learning_rate,
            momentum=opt.momentum,
            dampening=opt.dampening,
            weight_decay=opt.weight_decay,
            nesterov=False)
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer, 'min', patience=opt.lr_patience)

    if not opt.no_val:
        video_transform = transforms.Compose([
            transforms.ToTensor(opt.video_norm_value)])     

        validation_data = get_validation_set(opt, spatial_transform=video_transform)

        val_loader = torch.utils.data.DataLoader(
            validation_data,
            batch_size=opt.batch_size,
            shuffle=False,
            num_workers=opt.n_threads,
            pin_memory=True)

        val_logger = Logger(
                os.path.join(opt.result_path, 'val'+str(fold)+'.log'), ['epoch', 'loss', 'prec1', 'prec5'])
        test_logger = Logger(
                os.path.join(opt.result_path, 'test'+str(fold)+'.log'), ['epoch', 'loss', 'prec1', 'prec5'])


    best_prec1 = 0
    best_loss = 1e10
    if opt.resume_path:
        print('loading checkpoint {}'.format(opt.resume_path))
        checkpoint = torch.load(opt.resume_path)
        assert opt.arch == checkpoint['arch']
        best_prec1 = checkpoint['best_prec1']
        opt.begin_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])

    for i in range(opt.begin_epoch, opt.n_epochs + 1):

        if not opt.no_train:
            adjust_learning_rate(optimizer, i, opt)
            train_epoch(i, train_loader, model, criterion, optimizer, opt,
                        train_logger, train_batch_logger)
            state = {
                'epoch': i,
                'arch': opt.arch,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'best_prec1': best_prec1
                }
            save_checkpoint(state, False, opt, fold)

        if not opt.no_val:

            validation_loss, prec1 = val_epoch(i, val_loader, model, criterion, opt,
                                        val_logger)
            is_best = prec1 > best_prec1
            best_prec1 = max(prec1, best_prec1)
            state = {
            'epoch': i,
            'arch': opt.arch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_prec1': best_prec1
            }

            save_checkpoint(state, is_best, opt, fold)


    if opt.test:

        test_logger = Logger(
                os.path.join(opt.result_path, 'test'+str(fold)+'.log'), ['epoch', 'loss', 'prec1', 'prec5'])

        video_transform = transforms.Compose([
            transforms.ToTensor(opt.video_norm_value)])

        test_data = get_test_set(opt, spatial_transform=video_transform) 

        #load best model
        best_state = torch.load('%s/%s_best' % (opt.result_path, opt.store_name)+str(fold)+'.pth')
        model.load_state_dict(best_state['state_dict'])

        test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=opt.batch_size,
            shuffle=False,
            num_workers=opt.n_threads,
            pin_memory=True)

        test_loss, test_prec1 = val_epoch(10000, test_loader, model, criterion, opt,
                                        test_logger)

        with open(os.path.join(opt.result_path, 'test_set_bestval'+str(fold)+'.txt'), 'a') as f:
                f.write('Prec1: ' + str(test_prec1) + '; Loss: ' + str(test_loss))
        test_accuracies.append(test_prec1) 


with open(os.path.join(opt.result_path, 'test_set_bestval.txt'), 'a') as f:
    f.write('Prec1: ' + str(np.mean(np.array(test_accuracies))) +'+'+str(np.std(np.array(test_accuracies))) + '\n')

Namespace(annotation_path='ravdess_preprocessing/annotations.csv', arch='multimodalcnn', batch_size=8, begin_epoch=1, dampening=0.9, dataset='RAVDESS', device='cuda', fusion='ia', learning_rate=0.04, lr_patience=10, lr_steps=[40, 55, 65, 70, 200, 250], manual_seed=1, mask='softhard', model='multimodalcnn', momentum=0.9, n_classes=8, n_epochs=100, n_threads=16, no_train=False, no_val=False, num_heads=1, pretrain_path='EfficientFace_Trained_on_AffectNet7.pth.tar', result_path='results', resume_path='', sample_duration=15, sample_size=224, store_name='RAVDESS_multimodalcnn_15', test=True, test_subset='test', video_norm_value=255, weight_decay=0.001)
Initializing efficientnet
Total number of trainable parameters:  1854766


ValueError: too many values to unpack (expected 8)